In [1]:
import pyspark
from pyspark import SparkContext, StorageLevel
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark import SparkFiles
from datetime import datetime
from time import time


path = "../data/conditions_truncated.csv"

conf = SparkConf()
conf.getAll()

sc = SparkContext(appName="test")
    
spark = SparkSession(sc)
sc.setLogLevel("ERROR")

textfile = sc.textFile(path)

sc.addFile(path)


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/09 22:39:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [27]:
def get_support_value(diseases, diseases_support):
    disease = diseases.split(",")[0]
    print(disease)
    return diseases_support.filter(lambda x: x[0] == disease).collect()

def get_prob_value(diseases, baskets):
    disease = diseases.split(",")[1]
    return len(baskets.filter(lambda x: disease in x[1]).collect())/baskets.count()

def get_std_lift(support_x, prob_y, lift, total_baskets):
    max_value = max(support_x + prob_y - 1, 1/total_baskets)
    constant = (support_x * prob_y)
    numerator = lift - max_value / constant
    denominator = 1 / constant - max_value / constant
    return numerator/denominator

In [9]:
SUPPORT = 1000
K = 2

def build_pairs(basket, filtered_diseases):
    """
    Returns a list of bigrams that only contain diseases from the filtered list.
    
    Parameters:
    - basket: a tuple representing a basket of items
    - filtered_diseases: a list of diseases above the support treshold to include in the bigrams
    
    Returns:
    - a list of bigrams, where each bigram is a tuple of two diseases
    """
    basket_diseases = basket[1] # list of diseases in the current basket
    bigrams = []

    for i, disease_1 in enumerate(basket_diseases):
        # if the current disease is not in the list of filtered diseases, skip
        if disease_1 not in filtered_diseases:
            continue

        for disease_2 in basket_diseases[i+1:]:
        # if the current disease is not in the list of filtered diseases, skip
            if disease_2 not in filtered_diseases:
                continue

            # create a bigram from the two diseases and append it to the list of bigrams
            bigram = f"{disease_1},{disease_2}"
            bigrams.append((bigram, 1))
    
    return bigrams



before_baskets =  textfile.map(lambda line: line.split(",")) \
                        .map(lambda pair: (pair[2],[pair[4]])) \
                            .reduceByKey(lambda a,b: a + b)

baskets = before_baskets.mapValues(lambda x: sorted(set(x)))

printable = baskets.collect()

first_support = baskets.flatMap(lambda basket: [(disease, 1) for disease in basket[1]]) \
                            .reduceByKey(lambda a,b: a + b) \
                                .filter(lambda line: line[1] > SUPPORT)

printable2 = first_support.collect()

filtered_diseases = first_support.map(lambda line: line[0]).collect()

bigrams_support = baskets.flatMap(lambda basket: build_pairs(basket, filtered_diseases)) \
                                .reduceByKey(lambda a,b: a + b) \
                                    .filter(lambda line: line[1] > SUPPORT)

printable3 = bigrams_support.sortBy(lambda line: line[1], False).collect()

rules = {key:[value] for key, value in bigrams_support.collect()}
list_rules = []

# get association rules (X) -> (Y)
for key in rules:
    print(key)
    # get support of X
    support_x = get_support_value(key, first_support)[0][1]

    # confidence: support(X U Y)/support(X)
    confidence = rules[key][0] /  support_x
    rules[key][0] = confidence

    # get probability of Y
    prob_y = get_prob_value(key, baskets)

    # interest: confidence - prob(Y)
    interest = confidence - prob_y
    rules[key].append(interest)

    # lift: confidence / prob(Y)
    lift = confidence / prob_y
    rules[key].append(lift)

    # standard lift
    std_lift = get_std_lift(support_x, prob_y, lift, baskets.count())
    rules[key].append(std_lift)

    list_rules.append([key] + rules[key])

    #print(rules[key])

#print(rules)
x = sc.parallelize(list_rules)
x = x.filter(lambda line: line[4] > 0.2).sortBy(lambda line: line[4])

format_time = datetime.now().strftime("%Y-%m-%dT%H:%M:%S")
x.saveAsTextFile("{0}/Association Rules {1}".format("../results", format_time))

printable5 = x.collect()

print(printable5)


195662009,444814009
195662009,446096008
195662009,72892002
283371005,444814009
284551006,444814009
284551006,72892002
446096008,72892002
10509002,44465007
10509002,444814009
10509002,72892002
195662009,307731004
307731004,44465007
307731004,444814009
307731004,72892002
44465007,72892002
162864005,24079001
162864005,55822004
162864005,65966004
55822004,65966004
162864005,64859006
40055000,59621000
40055000,70704007
444814009,70704007
59621000,64859006
10509002,156073000
10509002,19169002
10509002,55822004
10509002,75498004
10509002,82423001
156073000,15777000
156073000,19169002
156073000,82423001
15777000,19169002
15777000,55822004
15777000,72892002
15777000,82423001
19169002,40055000
19169002,444814009
19169002,75498004
40055000,55822004
40055000,75498004
444814009,55822004
55822004,72892002
72892002,82423001
75498004,82423001
195662009,283385000
195662009,65363002
43878008,65363002
10509002,43878008
43878008,444814009
43878008,58150001
44465007,58150001
198992004,284549007
198992004,4

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/Users/eduardosantos/.pyenv/versions/3.9.14/envs/first-mdle-assignment/lib/python3.9/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/Users/eduardosantos/.pyenv/versions/3.9.14/envs/first-mdle-assignment/lib/python3.9/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/Users/eduardosantos/.pyenv/versions/3.9.14/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


192127007,72892002


KeyboardInterrupt: 

Exception in thread "serve RDD 8130" java.net.SocketTimeoutException: Accept timed out
	at java.net.PlainSocketImpl.socketAccept(Native Method)
	at java.net.AbstractPlainSocketImpl.accept(AbstractPlainSocketImpl.java:409)
	at java.net.ServerSocket.implAccept(ServerSocket.java:560)
	at java.net.ServerSocket.accept(ServerSocket.java:528)
	at org.apache.spark.security.SocketAuthServer$$anon$1.run(SocketAuthServer.scala:64)


In [3]:
def get_support_value(diseases, diseases_support):
    disease = diseases.rfind(",")
    key = diseases[:disease]
    return diseases_support.filter(lambda x: x[0] == key).collect()

def get_prob_value(diseases, baskets):
    disease = diseases.split(",")[1]
    return len(baskets.filter(lambda x: disease in x[1]).collect())/baskets.count()

def get_std_lift(support_x, prob_y, lift, total_baskets):
    max_value = max(support_x + prob_y - 1, 1/total_baskets)
    constant = (support_x * prob_y)
    numerator = lift - max_value / constant
    denominator = 1 / constant - max_value / constant
    return numerator/denominator



In [4]:
def build_trios(basket, filtered_diseases, filtered_diseases2):
    """
    Returns a list of trigrams that contain diseases from the filtered list.
    
    Parameters:
    - basket: a tuple representing a basket of items
    - filtered_diseases: a list of diseases above the support treshold to include in the trigrams
    - filtered_diseases2: a list of bigrams to include in the trigrams
    
    Returns:
    - a list of trigrams, where each trigram is a tuple of three diseases
    """
    diseases = basket[1] # list of diseases in the current basket
    trigrams = []
    
    for i, desease1 in enumerate(diseases):
        if desease1 not in filtered_diseases:
            continue
        for j, desease2 in enumerate(diseases[i+1:], i+1):
            # if the current disease is not in the list of filtered diseases, skip
            if desease2 not in filtered_diseases:
                continue

            # create a bigram from the two diseases and check if it is in the 
            # list of filtered bigrams
            if f"{desease1},{desease2}" not in filtered_diseases2:
                continue

            for desease3 in diseases[j+1:]:
                # if the current disease is not in the list of filtered diseases, 
                # skip to the next iteration
                if desease3 not in filtered_diseases:
                    continue
                
                # check if the two bigrams that can be created from the trigram 
                # are in the list of filtered bigrams
                if f"{desease1},{desease3}" not in filtered_diseases2 \
                    or f"{desease2},{desease3}" not in filtered_diseases2:
                    continue

                # create a trigram from the three diseases and append it to the 
                # list of trigrams
                trigrams.append((f"{desease1},{desease2},{desease3}", 1))
    return trigrams

In [10]:
filtered_diseases2 = bigrams_support.map(lambda line: line[0]).collect()

trigrams_support = baskets.flatMap(lambda basket: 
                                   build_trios(
                                        basket, 
                                        filtered_diseases, 
                                        filtered_diseases2)
                                        ).reduceByKey(lambda a,b: a + b) \
                                            .filter(lambda line: line[1] > SUPPORT)

printable4 = trigrams_support.sortBy(lambda line: line[1], False).collect()

association_rules = {key:[value] for key, value in trigrams_support.collect()}
rules_list = []

# get association rules (X,Y) -> (Z)

for key in association_rules:
    # get support of X U Y
    support_xy = get_support_value(key, bigrams_support)[0][1]

    # confidence: support(X U Y U Z)/support(X U Y)
    confidence = association_rules[key][0] /  support_xy
    association_rules[key][0] = confidence

    # get probability of Z
    prob_z = get_prob_value(key, baskets)

    # interest: confidence - prob(Z)
    interest = confidence - prob_z
    association_rules[key].append(interest)

    # lift: confidence / prob(Z)
    lift = confidence / prob_z
    association_rules[key].append(lift)

    # standard lift
    std_lift = get_std_lift(support_xy, prob_z, lift, baskets.count())
    association_rules[key].append(std_lift)

    rules_list.append([key] + association_rules[key])

x = sc.parallelize(rules_list)
x = x.filter(lambda line: line[4] > 0.2).sortBy(lambda line: line[4])

#format_time = datetime.now().strftime("%Y-%m-%dT%H:%M:%S")
#x.saveAsTextFile("{0}/Association Rules {1}".format("../results", format_time))

printable6 = x.collect()

print(printable6)

#for row in printable4:
#    print(row)
#
#print(len(printable4))

ERROR:root:KeyboardInterrupt while sending command.                             
Traceback (most recent call last):
  File "/Users/eduardosantos/.pyenv/versions/3.9.14/envs/first-mdle-assignment/lib/python3.9/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/Users/eduardosantos/.pyenv/versions/3.9.14/envs/first-mdle-assignment/lib/python3.9/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/Users/eduardosantos/.pyenv/versions/3.9.14/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

Exception in thread "serve RDD 10090" java.net.SocketTimeoutException: Accept timed out
	at java.net.PlainSocketImpl.socketAccept(Native Method)
	at java.net.AbstractPlainSocketImpl.accept(AbstractPlainSocketImpl.java:409)
	at java.net.ServerSocket.implAccept(ServerSocket.java:560)
	at java.net.ServerSocket.accept(ServerSocket.java:528)
	at org.apache.spark.security.SocketAuthServer$$anon$1.run(SocketAuthServer.scala:64)


In [ ]:
sc.stop()